In [8]:
import cv2
import matplotlib.pyplot as plt
import dlib
from imutils import face_utils
import math
import os
import numpy as np
import operator

In [9]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [10]:
def euclid_distance(x,y):
    if (len(x) != len(y)):
        return None
    sq=0
    for i in range(len(x)):
        sq += (x[i] - y[i]) * (x[i] - y[i])
    return sq ** 0.5    

In [11]:
def get_vector(img):
    image = cv2.imread(img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    for rect in rects:
    # xác định facial landmarks for the face region sau đó convert các facial landmarks (x,y) về numpy array, mỗi hàng là một cặp tọa độ
        shape = predictor(gray, rect)   

        # chuyển về dạng numpy các coordinates
        shape = face_utils.shape_to_np(shape)   # numpy array (68, 2)

        # Chuyển dlib's rectange (left, top, right, botttom) = (xmin, ymin, xmax, ymax) to OpenCV-style bounding box (xmin, ymin, w, h)
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 1)    # vẽ rectangle quanh khuôn mặt

        # duyệt qua các coordinates of facial landmarks (x, y) và vẽ chúng lên ảnh
        for (i,( x, y)) in enumerate(shape):
            cv2.circle(image, (x, y), 1, (255, 255, 255), -1)
            cv2.putText(image,str(i),(x-10,y-10),cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1)
    embedding=[]   
    m0=np.max(shape,axis=0)
    m1=np.min(shape,axis=0)
    width=euclid_distance(shape[0],shape[16])
    height=m0[1]-m1[1]
    # Tính khoảng cách các điểm đến trung tâm mặt 
    embedding.append(euclid_distance(shape[36],shape[30])/width)
    embedding.append(euclid_distance(shape[39],shape[30])/width)
    embedding.append(euclid_distance(shape[42],shape[30])/width)
    embedding.append(euclid_distance(shape[45],shape[30])/width)
    embedding.append(euclid_distance(shape[48],shape[30])/width)
    embedding.append(euclid_distance(shape[54],shape[30])/width)
    
    #Tính kích thước mắt, mũi, miệng
    righteye=euclid_distance(shape[42],shape[45])/width
    lefteye=euclid_distance(shape[36],shape[39])/width
    mouth=euclid_distance(shape[48],shape[54])/width
    nose_w=euclid_distance(shape[31],shape[35])/width
    nose_h=euclid_distance(shape[30],shape[27])/height
    
    embedding.append(righteye)
    embedding.append(lefteye)
    embedding.append(mouth)
    embedding.append(nose_h)
    embedding.append(nose_w)
    
    return embedding

In [12]:
a1="data\\Asian_16_1.png"
a1=get_vector(a1)
v={}
for filename in os.listdir('data'):
    print(filename,end=" ")
    try:
        vec=get_vector('data//'+filename)
        print(euclid_distance(a1,vec),end=" ")
        print(vec)
        v[filename]=euclid_distance(a1,vec)
    except:
        print("x")



African_0_0.png 0.3415048920094844 [0.3935570044931941, 0.22457786232857696, 0.25269424982970784, 0.43886883836214513, 0.2944926220362159, 0.33412198540858645, 0.21556530677961344, 0.2074197182600428, 0.5239092611837673, 0.24663414679817536, 0.25709622260182385]
African_0_1.png 0.3575106326070703 [0.4364891447871096, 0.24663171532705425, 0.142438680092441, 0.27323903662890714, 0.42829929721515, 0.31779536774209693, 0.17698409049755612, 0.20390037740699418, 0.5175356682827815, 0.18766228352850092, 0.25654677287771693]
African_10_0.png 0.270204227453731 [0.40470762173962643, 0.2651755284992312, 0.2881284777760449, 0.4380473867698075, 0.19302061743044752, 0.2425219954832664, 0.19219397287202264, 0.1885534195205361, 0.3420208092995673, 0.3099732441652, 0.21207153944219304]
African_10_1.png 0.2318099305022883 [0.3503236102385855, 0.22847177390934642, 0.28650998945749057, 0.4447676601724956, 0.1851398166982317, 0.287982427366207, 0.1905807459442273, 0.17460757394239457, 0.3320650008814185, 0

In [18]:
fig = plt.figure(figsize=(300, 300))
columns = 4
rows = 1
i=0
for key,val in sorted(v.items(), key=operator.itemgetter(1))[:4]:
    print (key,val)
    i+=1
    fig.add_subplot(rows, columns, i)
    img=cv2.imread("data\\"+key)
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show

Asian_16_1.png 0.0
Caucasian_10_1.png 0.03980189770795215
Asian_10_1.png 0.04984462453734556
Asian_15_1.png 0.05401623589731536


<function matplotlib.pyplot.show(close=None, block=None)>